# Lab 3 - Machine Learning with Spark

##### "A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P if its performance at tasks in T, as measured by P, improves with experience E"
-Tom M. Mitchell

#### Machine Learning - the science of getting computers to act without being explicitly programmed

MLlib is Spark’s machine learning (ML) library. Its goal is to make practical machine learning scalable and easy. It consists of common learning algorithms and utilities, including classification, regression, clustering, collaborative filtering (this example!), dimensionality reduction, as well as lower-level optimization primitives and higher-level pipeline APIs.

It divides into two packages:
- spark.mllib contains the original API built on top of RDDs.
- spark.ml provides higher-level API built on top of DataFrames for constructing ML pipelines.


Using spark.ml is recommended because with DataFrames the API is more versatile and flexible. But we will keep supporting spark.mllib along with the development of spark.ml. Users should be comfortable using spark.mllib features and expect more features coming.

http://spark.apache.org/docs/latest/mllib-guide.html

## Online Purchase Recommendations

Learn how to create a recommendation engine using the Alternating Least Squares algorithm in Spark's machine learning library

<img src='https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/ALS.png' width="70%" height="70%"></img>

### The data

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.  The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

http://archive.ics.uci.edu/ml/datasets/Online+Retail

<img src='https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/FullFile.png' width="80%" height="80%"></img>

# <span style="color:blue">Section 0: </span> <br>Obtaining the dataset and performing initial prep work

### Download the data
Type (or copy) the following in the cell below:<br>
<span style="color:red">!rm 'OnlineRetail.csv.gz' -f<br>
!wget https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/OnlineRetail.csv.gz</span>

### Load the csv into an RDD (at first, each row in the RDD is a string which corresponds to a line in the csv)
Type (or copy) the following in the cell below:<br>
loadRetailData = sc.textFile("./OnlineRetail.csv.gz")<br>
print loadRetailData.take(2)

## Prepare and shape the data:  "80% of a data science project"

### Remove the header from the RDD and split the string in each row by comma<br>
<div class="panel-group" id="accordion-11">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-11" href="#collapse1-11">
        Hint</a>
      </h4>
    </div>
    <div id="collapse1-11" class="panel-collapse collapse">
      <div class="panel-body">Type (or copy) the following in the cell below: <br>
          header = loadRetailData.first()<br>
          loadRetailData = loadRetailData.filter(lambda line: line != header).map(lambda l: l.split(","))<br>

      print loadRetailData.take(2)
      </div>
    </div>
  </div>

##### NOTE:  The original file at UCI's Machine Learning Repository has commas in the product description.  Those have been removed to speed up the lab.
### Only keep rows that have a purchase quantity that is greater than 0, a customerID not equal to 0, and a non blank stock code after removing non-numeric characters

<div class="panel-group" id="accordion-12">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-12" href="#collapse1-12">
        Hint</a>
      </h4>
    </div>
    <div id="collapse1-12" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below: <br>
<span style="color:red"> import re </span><br>
loadRetailData = loadRetailData.filter(lambda l: int(l[3]) > 0\ <br>
                                and len(re.sub("\D", "", l[1])) != 0 \ <br>
                                and len(l[6]) != 0) <br>
print loadRetailData.take(2)
      
      </div>
    </div>
  </div>

### Map each line to a row and create a data frame 
<span style="color:red">Run the following cell:</span>

In [ ]:
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

#Convert each line to a Row.
loadRetailData = loadRetailData.map(lambda l: Row(inv=int(l[0]),\
                                    stockCode=int(re.sub("\D", "", l[1])),\
                                    description=l[2],\
                                    quant=int(l[3]),\
                                    invDate=l[4],\
                                    price=float(l[5]),\
                                    custId=int(l[6]),\
                                    country=l[7]))

# Infer the schema, and register the DataFrame as a table.
retailDf = sqlContext.createDataFrame(loadRetailData)
print retailDf.printSchema()

retailDf.registerTempTable("retailPurchases")
print sqlContext.sql("SELECT * FROM retailPurchases limit 2").toPandas()

### DATA EXTRACTION FOR THE RECOMMENDATION ENGINE
#### Use an SQL statement (between the three double quotes) to extract from the reatailDf dataframe the columns which are needed for the recommendation engine: custId, stockCode and the customer's preference for a product, which is not available explicitly, so it will be "approximated" by the number of times a customer buys a product (<span style="color:blue">quantity</span>).
#### Note as well that the recommendation engine expects an integer datatype for both the customer id and the stock code, and a double value for the customer ranking or preference. The columns are therefore casted to the proper datatype as part of the SQL statement.

<div class="panel-group" id="accordion-13">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-13" href="#collapse1-13">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-13" class="panel-collapse collapse">
      <div class="panel-body"> 
      Inject the following SQL string between the triple quotes in the cell below:<br>
<span style="color:red">SELECT <br>
    cast(custId as integer), cast(stockCode as integer), cast(count(*) as double) as preference <br>
FROM <br>
    retailPurchases <br>
group <br>
    by custId, stockCode </span>
      </div>
    </div>
  </div>

In [ ]:
query = """
"""
retailDf = sqlContext.sql(query)

retailDf.show(10)

### Randomly split the data into a testing set (20% of the data), and a training set (80% of the data)

<div class="panel-group" id="accordion-14">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-14" href="#collapse1-14">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-14" class="panel-collapse collapse">
      <div class="panel-body"> 
<span style="color:blue">Type (or copy) the following in the cell below:</span><br>
testDf, trainDf = retailDf.randomSplit([.2,.8],1)<br><br>

print testDf.take(2)<br>
print trainDf.take(2)
      </div>
    </div>
  </div>

# End of <span style="color:blue">Section 0 </span>.

### In the rest of this tutorial, we will build different recommendation engines, following an identical approach but using a few different options / alternatives offered by the Apache Spark libraries MLlib and Spark ML. 
### Examples of the alternatives which will be explored include using the original RDD based Spark MLlib library, versus the more recent (dataframe based) Spark ML, as well as comparing results when using the option which differentiates between EXPLICIT customer feedback (usually provided by rating a product), versus the more common IMPLICIT customer feedback (usually derived from customer behavior).

# <span style="color:red">Build recommendation models. We will have 4 sections:</span> 
## <span style="color:blue">Section 1-</span> Use the MLlib library assuming EXPLICIT customer feedback
## <span style="color:blue">Section 2-</span> Use the MLlib library assuming IMPLICIT customer feedback
## <span style="color:blue">Section 3-</span> Use the Spark ML library assuming EXPLICIT customer feedback
## <span style="color:blue">Section 4-</span> Use the Spark ML library assuming IMPLICIT customer feedback

# <span style="color:blue">Section 1 (Please ensure you've run Section 0 above)</span>
### Use MLlib library and (assume) EXPLICIT user feedback on product ratings.

### <span style="color:green">Section 1.1: Training the explicit MLlib model</span>

Run the content of the cell below:

In [ ]:
# Some imports
from pyspark.mllib.recommendation import ALS, Rating

# Convert the trainDf dataframe (defined above) to the underlying RDD. It should be noted that 
# these conversions have a cost... We will test shortly in a subsequent paragrapah whether the 
# MLlib algorithm can ingest directly the dataframe, which would be more convenient since data 
# frames can be queried using SQL or the Domain Specific Langue which is SQL-like.
trainRDD = trainDf.rdd

# We are casting below the elements of the RDD to "Rating" objects, corresponding to the
# input which is expected by the Spark ALS algorithm. The definition of this ALS 
# Rating class can be found fairly easily online and looks as follows:  
# Rating(user: Int, product: Int, rating: Double)
trainRDDAsRating = trainRDD.map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

# As it will also become apparent using the RDD below, the explicit casting of the ALS input 
# elements to Rating objects is not required, and keeping the original triplets with the 
# proper data types appears to be sufficient as well.
trainRDDNoRating = trainRDD.map(lambda l: (int(l[0]), int(l[1]), float(l[2])))

### Take a look at the first few elements of the RDDs defined above:
<div class="panel-group" id="accordion-15">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-15" href="#collapse1-15">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-15" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
trainRDDAsRating.first() 
      </div>
    </div>
  </div>

In [ ]:
# Take a peek at the first elements of the RDD made of "Rating" objects. As expected, the
# Rating class is made of the three fields mentioned above: user (integer), product (integer) 
# and rating (double)


<div class="panel-group" id="accordion-16">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-16" href="#collapse1-16">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-16" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
trainRDDNoRating.first()
      </div>
    </div>
  </div>

In [ ]:
# Also take a look at the first element of the RDD made from basic triplets.


Run the content of the cell below:

In [ ]:
# Prepare to train the model, using a basic choice of hyper parameters for the ALS algorithm.
rank = 5
numIterations = 15
alpha = 0.01
lambda1 = 0.01

# Train the model using the Rating class
model = ALS.train(trainRDD, rank, numIterations, 0.01, -1, False, 10)

# Train the model using the basic triplets instead of the "Rating" class. It can be verified
# that this works in the same way and produces the same model. We can replace "model" by 
# "modelNoRating" in the cells below which will not cause any difference in the results
modelNoRating = ALS.train(trainRDDNoRating, rank, numIterations, 0.01, -1, False, 10)

print "The model has been trained"

### <span style="color:green">Section 1.2: Building the test RDD</span>

#### Now that the ALS model has been trained, it needs to be tested for accuracy. Remember that the original dataset was split into a training and testing set. We will use the testing set for this purpose. The testing set has three fields: user, product, rating and we need to eliminate the third field (column) in order to produce an RDD of the format (user, product), which is done in the cell below. This RDD will be used by the prediction logic to produce a new RDD which has the prediction/recommendation column added to the original two.

Run the content of the cell below:

In [ ]:
# The testing set was obtained as a subset of the overall dataset, so it contains as well
# three fields, namely: user, product, rating.
testRDD = testDf.rdd

# testRDD0 is shaped to keep the original three fields: user, product, rating where the
# user and product are grouped into a tuple acting as the key in a (key, value) pair... The
# value being the rating. This RDD is going to be needed, once we get predictions from the
# model, to compare original user ratings with the ones produced by the model, in order to
# calculate its accuracy.
testRDD0 = testRDD.map(lambda Row: ((Row[0], Row[1]), Row[2]))

# testRDD1 below corresponds to the testRDD where the original user rating is eliminated. This
# RDD will be used as input for generating predictions
testRDD1 = testRDD.map(lambda Row: (Row[0], Row[1]))

# Verify the proper formatting of testRDD0 by taking a look at the first element(s)
testRDD0.take(2)

### <span style="color:green">Section 1.3: Getting predictions using the explicit MLlib model</span>

### Run predictAll on testRDD1 defined above, to obtain a set of predictions for each user / item pair

<div class="panel-group" id="accordion-17">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-17" href="#collapse1-17">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-17" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
predict = model.predictAll(testRDD1)
      </div>
    </div>
  </div>

### Take a look at the first few elements of the resulting predictions

<div class="panel-group" id="accordion-18">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-18" href="#collapse1-18">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-18" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
predict.take(5)
      </div>
    </div>
  </div>

In [ ]:
# The RDD named "predict" above now has the model generated ratings added to it. The closer those
# ratings are to the original user ratings we had in the testRDD, the more accurate the model
# will be.


### <span style="color:green">Section 1.4: Comparing user entries with predicted ratings and getting a Mean Squared Error</span>

Run the contents of the cell below:

In [ ]:
# Step 1.4.1
# The next task will consist in joining this new RDD of predictions with the original testRDD
# in order to compare original and generated ratings. We will therefore shape this new RDD
# as a (key, value) pair RDD where the key will correspond to a tuple consisting of the same
# two fields (user, product) as in the testRDD. This is achieved with the transformation below.
predict = predict.map(lambda r: ((r.user, r.product), r.rating))
predict.first()

### Execute of join of the RDD defined above (predict) with the original testRDD0 defined higher into a new RDD named "ratesAndPreds". You can also look at the first or first few elements of ratesAndPreds.

<div class="panel-group" id="accordion-19">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-19" href="#collapse1-19">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-19" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
ratesAndPreds = predict.join(testRDD0)<br>
ratesAndPreds.first() 
      </div>
    </div>
  </div>

In [ ]:
# Step 1.4.2
# We can now proceed with the joining of both RDDs, on the (user, product) field.


### Using RatesAndPreds, calculate the Mean Squared Error between original user ratings and the predictions
<div class="panel-group" id="accordion-20">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-20" href="#collapse1-20">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-20" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
MSE = ratesAndPreds.map(lambda l: (l[1][0]- l[1][1])**2).mean()<br>
print MSE  
      </div>
    </div>
  </div>

In [ ]:
# Step 1.4.3
# The resulting RDD, named ratesAndPreds, has therefore the pair (user, product) as its key,
# and the original and generated ratings as the values. We can therefore calculate the average
# of the difference between the two values as follows. (Mean Squared Error)


## An MSE value by itself is not a meaningful number as it needs to be compared with other values in an iterative process in order to identify the best hyper parameters for the ALS algorithm in this particular scenario. This can be left as an exercise...

### We can now perform the same steps performed above, but trying to manipulate dataframes whenever possible rather than RDDs, since joining dataframes can be more easily done using SQL syntax.<br><br><span style="color:blue">Note: We are still not using Spark.ml but still using the Spark MLlib library at this point.</span>

### <span style="color:green">Section 1.5: Training explicit MLlib model, working with dataframes</span>

Run the contents of the cell below:

In [ ]:
# Step 1.5.1
# Prepare and test a model where we pass the training dataframe directly, containing the three
# required fields for ALS: user, product, rating

from pyspark.mllib.recommendation import ALS, Rating
rank = 5
numIterations = 15
alpha = 0.01
lambda1 = 0.01

modeldf = ALS.train(trainDf, rank, numIterations, 0.01, -1, False, 10)

print "The model has been trained"

### Extract the (user, product) pair from the RDD testDf (defined higher), this time as a dataframe named "testDf2" and name the columns "user" and "product". Take a look at the first element of testDf2 to verify correctness of your logic.

<div class="panel-group" id="accordion-21">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-20" href="#collapse1-21">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-21" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
testDf2 = testDf.map(lambda l: (l[0], l[1])).toDF(["user", "product"])<br>
testDf2.first()  
      </div>
    </div>
  </div>

In [ ]:
# As performed in previous steps above, we will now extract the (user, product) pair from the 
# testDf and drop the rating column, since it gets added by the prediction logic.


### As performed higher in this lab, run predictAll on the elements of testDf2 and save the output in an RDD named "predictUsingDf". Remember that since we are still using MLlib, predictAll will expect an RDD but testDf2 is a dataframe...

<div class="panel-group" id="accordion-22">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-22" href="#collapse1-22">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-22" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
predictUsingDf = modeldf.predictAll(testDf2.rdd)  
      </div>
    </div>
  </div> 

In [ ]:
# Get now the predictions for testDf2. Note that we need to pass in the underlying RDD to
# predictAll. Passing in the DataFrame returns an error indicating that the method expects
# an RDD.
# Performance enhancement: It is not necessary (except for educational purposes in a tutorial)
# to make testDf2 a dataframe since predictAll below requires an RDD. Consequently, we can 
# probably avoid converting the result of the lambda transformation back to a dataframe...


#predictUsingDf is an RDD of "Rating" objects comprised of the fields: user, product, rating

### Take a look at the first element of predictUsingDf defined in the cell above:
<div class="panel-group" id="accordion-23">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-23" href="#collapse1-23">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-23" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
predictUsingDf.first()  
      </div>
    </div>
  </div> 

In [ ]:
# Take a look at the first element of predictUsingDf.


### predictUsingDf is an RDD (of Rating objects) which was returned by Spark in the cell above. This RDD has three elements: user, product and (predicted) rating. Let us now construct a dataframe from this RDD. The goal of constructing a dataframe is to be able to subsequently leverage SQL statements. Let the name of the new dataframe be "predictDf" and the three columns: "user", "product" and "rating".

<div class="panel-group" id="accordion-24">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-24" href="#collapse1-24">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-24" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
predictDf = predictUsingDf.map(lambda r: (r.user, r.product, r.rating)).\ <br>
map(lambda (a,b,c): Row(a,b,c)).toDF(["user", "product", "rating"])  
      </div>
    </div>
  </div> 

In [ ]:
# Now we will rebuild a data frame from the RDD predictUsingDf, so as to be able to use SQL to
# compare the original ratings with the ones generated by the recommendation engine.


### Look at the first 5 rows of the predictDf dataframe.
<div class="panel-group" id="accordion-25">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-25" href="#collapse1-25">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-25" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
predictDf.show(5)
      </div>
    </div>
  </div> 

In [ ]:
# Verify that the dataframe was built correctly by looking up a few elements.


### We will now proceed with the same approach used above, where the ratings generated by the recommendation engine and those originally available in the dataset are brought together in the same structure for comparison. The difference with the previous method is that we are going to use an SQL join instead of the RDD join, which is more convenient and also (potentially) more efficient.

**Write an SQL query between the three double quotes in the cell below, where:**<br>
1- The selected columns are: custId, stockCode, preference, user, product, rating<br>
2- The tables being joined are testDf and predictDf<br>
3- The join condition is on custId = user and stockCode = product

<div class="panel-group" id="accordion-26">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-26" href="#collapse1-26">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-26" class="panel-collapse collapse">
      <div class="panel-body"> 
SELECT <br>
    custId, stockCode, preference, user, product, rating<br>
FROM <br>
    testDf, predictDf <br>
WHERE <br>
    custId = user and stockCode = product
      </div>
    </div>
  </div> 

In [ ]:
# This dataframe contains the original rating value. Register it as a table.
testDf.registerTempTable("testDf")

# This dataframe contains the generated ratings. Register it as a table.
predictDf.registerTempTable("predictDf")

# Join both tables.
join = """
"""
joinDf = sqlContext.sql(join)

#Print a few rows from the join result
print joinDf.filter("preference >=3").orderBy("custId", "preference").show(10)

### Calculating the Mean Squared Error, and unsurprisingly, it is exactly the same value as previously calculated, since the same hyper parameters were used to train the model, and the usage of data frames versus RDDs does not affect the model which is produced by Spark.

<div class="panel-group" id="accordion-27">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-27" href="#collapse1-27">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-27" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following into the cell below:<br>
MSE = joinDf.map(lambda l: (l.preference - l.rating)**2).mean()<br>
print MSE
      </div>
    </div>
  </div> 

In [ ]:
# Compute the Mean Squared Error. Note that we can refer to the columns by name instead of by
# index...


### <span style="color:green">Section 1.6: Manual verification of ALS recommendations for one chosen customer</span>

### We will now work on verifying "manually" the efficiency of our model. In order to do so, we will pick one customer who bought a reasonably small, yet meaningful number of items (i.e more than one or two, but not dozens) and check the top three recommendations for that particular customer in order to decide whether these make sense or not. To begin, we write a SQL query to find customers ids who bought a reasonable number of items, i.e between 5 and 10. We start with a count of 7 items (feel free to update the query below with a different count for other attemps)

### Write between the three double quotes below an SQL statement to select customers who bought 7 items
<div class="panel-group" id="accordion-28">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-28" href="#collapse1-28">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-28" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) this SQL statement between the triple quotes below:<br>
  select custId, count(*) as count from retailPurchases <br>
           group by custId having count = 7 order by count 
      </div>
    </div>
  </div> 

In [ ]:
# The order by clause in the query below has been added in case the having clause is changed
# to an inequality rather than an equality (i.e something like having count > 5 ...)
query = """
"""

sqlContext.sql(query).take(10)

### From the results of the SQL query above, we arbitrarily pick one of the customers ids and inject it in a simple query in order to find out what items this person bought.

### Inject between the three double quotes below, a simple SQL statement to select items purchased by the chosen customer id
<div class="panel-group" id="accordion-29">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-29" href="#collapse1-29">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-29" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) this SQL statement between the triple quotes below (chosing customer Id 16248 from the list obtained above):<br>
    SELECT distinct stockCode, description from retailPurchases <br>
               where custId in (16248)
      </div>
    </div>
  </div> 

In [ ]:
sqlContext.sql(""" """).take(25)

### Get the top 3 recommendations for the user chosen above. Use the recommendProducts API. The Spark Python ALS set of APIs is available online, at the following link (among others): http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#module-pyspark.mllib.recommendation

<div class="panel-group" id="accordion-30">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-30" href="#collapse1-30">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-30" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
  modeldf.recommendProducts(16248,3)
      </div>
    </div>
  </div>   

#### Use a SQL query to quickly lookup the text description of the recommendations above. It can be noticed that some items use multiple text descriptions for the same stockCode (<span style="color:red">possible further data cleaning opportunity here...</span>), which is the reason we may want to select several rows from the resulting dataframe. 
#### <span style="color:green">There can be randomness injected in the recommendations, so it might not be possible to get the exact same results for consecutive runs...</span>

### Inject between the three double quotes below, an SQL query to look up the text descriptions of some (or all) of the top 3 recommend products found above.
<div class="panel-group" id="accordion-31">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-31" href="#collapse1-31">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-31" class="panel-collapse collapse">
      <div class="panel-body"> 
Write (or copy) this SQL statement between the triple quotes below:<br>
SELECT distinct stockCode, description from retailPurchases <br>
               where stockCode in (17012, 15056) 
      </div>
    </div>
  </div>      

In [ ]:
sqlContext.sql(""" """).take(35)

### <span style="color:green">Section 1.7: Analyzing the ALS recommendation engine for data (customer ids) which did not exist in the training set...</span>

#### Extra-credit activity: Remember that the original data set was randomly split between training and testing set (80% , 20%). This means that some users may be present in the testing set but not in the training set. Finding those users can be achieved in several different ways. Below is an example:

Write an SQL query between the triple quotes below, to identify the customer ids which are present in testDf but not present in trainDf. If you need to write more than one query, feel free to insert additional cells.
<div class="panel-group" id="accordion-32">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-32" href="#collapse1-32">
        Hint 1</a>
      </h4>
    </div>
    <div id="collapse1-32" class="panel-collapse collapse">
      <div class="panel-body"> 
Remember that the left outer join of two tables T1 and T2 on T1.c1=T2.c2 will return a NULL row associated to the row from T1, whenever that row from T1 does **not** have a match in T2
      </div>
    </div>
  </div> 
  
<div class="panel-group" id="accordion-33">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-33" href="#collapse1-33">
        Hint 2</a>
      </h4>
    </div>
    <div id="collapse1-33" class="panel-collapse collapse">
      <div class="panel-body"> 
Left outer join of testDf and trainDf on custId to determine customer Ids which exist in test but not in training:<br>
SELECT testDf.custId, trainDf.custId FROM<br>
testDf LEFT OUTER JOIN trainDf<br>
ON testDf.custId = trainDf.custId<br>
where isNull(trainDf.custId)
      </div>
    </div>
  </div>    

In [ ]:
#Find users which were in the testing set but not in the training set
trainDf.registerTempTable("trainDf")
testDf.registerTempTable("testDf")
query = """
"""
joinDf = sqlContext.sql(query)

joinDf.show(20)

### Let's pick one of the customer ids which was present in the testing set but not in the training set and get the top 3 recommendations. 
### The execution of the cell below should return an error due to the fact that the model never obtained data for a user which was not present in the training set. Consequently, it makes sense to cleanup the testing set by removing all users and products which were not present in the training set...

### Using a customer id which is found to be present in the test set but not in the training set, get the top 3 recommendations:
<div class="panel-group" id="accordion-34">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-34" href="#collapse1-34">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-34" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following into the cell below:<br>
modeldf.recommendProducts(16462,3)
      </div>
    </div>
  </div> 

### We can verify this in a different way, by comparing the number of entries which were fed into the model and the number of entries it produced. Those were respectively the RDDs / dataframes <span style="color:red">testDf2</span> and <span style="color:red">predictUsingDf</span> higher up in Section 1.5
### It can be noticed that some entries did not produce a recommendation and those correspond to customer ids which the trained model does not know about (also known as the <span style="color:red">cold start</span> problem: what to recommend for a new customer who has never provided any ratings or preferences).
### It can be noticed as well that the remarks made for customer ids can also be made for stock ids, and a best practice would be to ensure that the randomly generated test set is similar in distributions to the training set.

In [ ]:
testDf2.count()

In [ ]:
predictUsingDf.count()

### MLlib also provides a recommendFeatures API, which will return the top users for a given product such as <span style="color:red">Hand Warmer Owl Design</span>. Let's try it.

### In the same way we obtained higher the recommended products for a user, we can also obtain the recommended users for a product using recommendUsers, which is documented at the same link as described above for recommendProdcuts.
<div class="panel-group" id="accordion-35">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-35" href="#collapse1-35">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-35" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
    modeldf.recommendUsers(22865,3)
      </div>
    </div>
  </div> 

### This is an SQL query in which we can inject the customer ids to identify the products which were bought. A further analysis of the results is left as an exercise...

In [ ]:
sqlContext.sql("""SELECT distinct stockCode, description from retailPurchases 
                  where custId in (17841)""").take(35)

### <span style="color:green">Section 1.8: A Quick look under the hood..</span>

### MLlib gives access to the user and product features matrices mentioned at the beginning of this lab. The ratings for each user and product are obtained as a result of the dot product of the user vector and Product vector from these matrices.

### We can look at a row from the userFeatures matrix. We can select the features for the user we have been working with so far: <span style="color:red">16248</span>. Notice the number of features "5" will correspond to the <span style="color:red">rank</span> parameter which was used to train the model.

### Still using the Python ALS API documentation, use the "userFeatures()" API to print the vector representing user "16248" (for example).
<div class="panel-group" id="accordion-36">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-36" href="#collapse1-36">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-36" class="panel-collapse collapse">
      <div class="panel-body"> 
Type (or copy) the following in the cell below:<br>
 model.userFeatures().lookup(16248)
      </div>
    </div>
  </div>  

### Here is another view, taking the first two elements of the <span style="color:red">Features</span> matrix. We see the features associated with two products.

Type (or copy) the following in the cell below: <br>
model.productFeatures().take(2) 

## <span style="color:green">Additional work will be suggested at the end of this lab in manipulating these user and product features in more detail...</span>

# <span style="color:blue">Section 2 (Please ensure you've run Section 0 above)</span>
### Use MLlib library and (assume) IMPLICIT user feedback on product ratings.
### We will continue using data frames for convenience.

### In implicit feedback rating, the ALS algorithm will adapt its internal weights to treat the provided user ratings as being derived through some process rather than being directly given by the user. This is closer to reality in our example, since the number of items purchased by the user were used as a form of representing the user preference for that item.<br> <br>The general flow of the approach will however remain identical to the one in Section 1 above.

### <span style="color:green">Section 2.1: Training the implicit MLlib model.</span>

In [ ]:
# Working with Implicit Feedback (We will treat the number of times a customer bought an item
# as implicit feedback by using trainImplicit)
from pyspark.mllib.recommendation import ALS, Rating
rank = 5
numIterations = 10
alpha = 0.01
lambda1 = 0.01
modelImplicit1 = ALS.trainImplicit(trainDf, rank, numIterations, alpha, -1, lambda1, False, 10)
print "modelImplicit1 has been trained"

### A second model with different hyper parameter values is trained for comparison purposes.

In [ ]:
rank = 15
numIterations = 20
alpha = 0.01
lambda1 = 0.01
modelImplicit2 = ALS.trainImplicit(trainDf, rank, numIterations, alpha, -1, lambda1, False, 10)
print "modelImplicit2 has been trained"

### <span style="color:green">Section 2.2: In a similar way to what was done higher in Section 1, we build a test dataframe which only has two columns: user and product. The rating column is added by the recommendation engine.</span>

Following examples from Section 1 above, fill out the code in the cell below:<br>
<div class="panel-group" id="accordion-37">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-37" href="#collapse1-37">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-37" class="panel-collapse collapse">
      <div class="panel-body"> 
testDf2 = testDf.map(lambda l: (l[0], l[1])).toDF(["user", "product"])<br>
testDf2.first()
      </div>
    </div>
  </div>  

In [ ]:
#Extract the (user, product) pair from the testDf and drop the rating


### <span style="color:green">Section 2.3: Getting predictions using the implicit MLlib model</span>

Following examples from Section 1 above, fill out the code in the cell below:
<div class="panel-group" id="accordion-38">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-38" href="#collapse1-38">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-38" class="panel-collapse collapse">
      <div class="panel-body"> 
predictImplicit = modelImplicit1.predictAll(testDf2.rdd)
      </div>
    </div>
  </div>

In [ ]:
# Get now the predictions for testDf2. Note that we need to pass in the underlying RDD to
# predictAll. Passing in the DataFrame returns an error indicating that the method expects
# an RDD.

#predictImplicit is an RDD of "Rating" objects comprised of the fields: user, product, rating

Depending on how you named your RDD in the cell above, either run this cell as is or update it to run properly...

In [ ]:
predictImplicit.first()

### Note as well that in the case of implict ratings logic, the ratings returned by the recommendation engine are values between 0 and 1, as they correspond to a "confidence" level rather than explicit rating value.

Following examples from Section 1 above, fill out the code in the cell below to build a predictImplicitDf dataframe (from the predictImplict RDD). The dataframe should have three columns "user", "product" and "rating":
<div class="panel-group" id="accordion-39">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-39" href="#collapse1-39">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-39" class="panel-collapse collapse">
      <div class="panel-body"> 
predictImplicitDf = predictImplicit.map(lambda r: (r.user, r.product, r.rating)).\<br>
map(lambda (a,b,c): Row(a,b,c)).toDF(["user", "product", "rating"])
      </div>
    </div>
  </div>

In [ ]:
# Now we will rebuild a data frame from the RDD predictImplicit, so as to be able to use SQL.


Depending on how you named your dataframe in the cell above, either run this cell as is, or update it to run properly...

In [ ]:
predictImplicitDf.show(5)

### <span style="color:green">Section 2.4: Comparing user entries with predicted ratings and getting a Mean Squared Error</span>

### In the cell below, we will join the test table with the recommendation engine predictions into one single table / dataframe, which will allow us to calculate the accuracy of the algorithm by computing the deltas between original and computed user ratings.

Following the example provided above in Section 1, inject an SQL query between the triple quotes in the cell below to JOIN tables testTable and predictImplicitTable on the customer id and stockCode columns.
<div class="panel-group" id="accordion-40">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-40" href="#collapse1-40">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-40" class="panel-collapse collapse">
      <div class="panel-body"> 
SELECT <br>
    custId, stockCode, preference, user, product, rating<br>
FROM <br>
    testTable, predictImplicitTable<br> 
WHERE<br>
    custId = user and stockCode = product
      </div>
    </div>
  </div>

In [ ]:
testDf.registerTempTable("testTable")
predictImplicitDf.registerTempTable("predictImplicitTable")
join = """
"""
joinDf = sqlContext.sql(join)

print joinDf.filter("preference >=3").orderBy("custId", "preference").show(10)

Following the example from Section 1 above, calculate the MSE for your first model (modelImplicit1) in the cell below and print it.
<div class="panel-group" id="accordion-41">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-41" href="#collapse1-41">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-41" class="panel-collapse collapse">
      <div class="panel-body"> 
MSE = joinDf.map(lambda l: (l.preference - l.rating)**2).mean()<br>
print MSE
      </div>
    </div>
  </div>

In [ ]:
#Calculate the Mean Squared Error for modelImplicit1.


### In order to calculate the MSE for the second model, we need to rerun the few cells above by switching one model name for the other.

Revisit the few cells above from this Section 2, and adapt them to inject the SECOND model (modelImplicit2) and derive its MSE as well. Compare the values...As an alternative, you may also want to duplicate the cells with the second model (modelImplicit2) to have both available simultaneously.
<div class="panel-group" id="accordion-42">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-42" href="#collapse1-42">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-42" class="panel-collapse collapse">
      <div class="panel-body"> 
After having changed the cells above to deal with the second model, we can reuse the same statement:<br>
MSE = joinDf.map(lambda l: (l.preference - l.rating)**2).mean()<br>
print MSE
      </div>
    </div>
  </div>

In [ ]:
#Calculate the Mean Squared Error for modelImplicit2.


### Confirm that based on the results above, it seems that modelImplict2 has a lower MSE and therefore has better accuracy. Consequently, we use model2 in the remainder of this section.
### Note: For Implicit feedback models, since the computed ratings are always between 0 and 1, it is not customary to use a Mean Squared Error approach, but rather an Area Under the Curve. The various types of evaluations of recommendation engines are outside of the limited scope of this lab and can be found in the literature such as in this article: http://users.csc.calpoly.edu/~dekhtyar/466-Fall2010/lectures/lec11-1.466.pdf . We are here limiting ourselves to a straightforward MSE evaluation due to time limitations.

### <span style="color:green">Section 2.5: Manual verification of ALS recommendations for one chosen customer...</span>

Following the example from Section 1 above, and using modelImplicit2, find the top three recommended products for one chosen customer (we can keep the same customer id as in the previous section: 16248):
<div class="panel-group" id="accordion-43">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-43" href="#collapse1-43">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-43" class="panel-collapse collapse">
      <div class="panel-body"> 
modelImplicit2.recommendProducts(16248,3)
      </div>
    </div>
  </div>

In [ ]:
# The example below keeps the same customer as the one picked in Section 1 above.


Following the example from Section 1 above, write an SQL statement in the cell below to look up the textual description of items purchased by customer id 16248.
<div class="panel-group" id="accordion-44">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-44" href="#collapse1-44">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-44" class="panel-collapse collapse">
      <div class="panel-body"> 
SELECT distinct stockCode, description from retailPurchases <br>
               where custId in (16248)
      </div>
    </div>
  </div>

In [ ]:
#Reminder of what this customer bought...


Following the example from Section 1 above, write an SQL statement in the cell below to look up the textual description of recommended items for customer id 16248...
<div class="panel-group" id="accordion-45">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-45" href="#collapse1-45">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-45" class="panel-collapse collapse">
      <div class="panel-body"> 
Remember that the exact stock codes may have to be updated depending on your choice for the customer:<br>
SELECT distinct stockCode, description from retailPurchases <br>
 where stockCode in (22111, 23355, 22865)
      </div>
    </div>
  </div>

In [ ]:
#Here is what we are recommending for them
sqlContext.sql(""" """).take(35)

## Do the recommendations look like reasonable suggestions ? How do they compare to the Explicit feedback model ?

# In the following section, we will turn our attention to the more recent Spark ML library which is built on top of data frames.

# <span style="color:blue">Section 3 (Please ensure you've run Section 0 above)</span>
### Use SPARK ML library and (assume) EXPLICIT user feedback on product ratings.
### SPARK ML uses data frames.

### <span style="color:green">Section 3.1: Training the explicit Spark ML models</span>

### The syntax of Spark ML is slightly different than that of Spark MLlib. The model is trained here by "fitting" the algorithm to the provided input data frame, using the now familiar hyper parameters: rank and maxIter. Also note that the columns of interest (to ALS logic) in the input dataframe are selected by name using the 'userCol', 'itemCol' and 'ratingCol' keywords.

Run the section below which trains two different models using Spark ML...

In [ ]:
# We train two models using different combinations for the rank and the number of iterations.
from pyspark.ml.recommendation import ALS

als1 = ALS(rank=15, maxIter=15,userCol="custId",itemCol="stockCode",ratingCol="preference")
model1 = als1.fit(trainDf)

als2 = ALS(rank=2, maxIter=10,userCol="custId",itemCol="stockCode",ratingCol="preference")
model2 = als2.fit(trainDf)

print "The models have been trained"

### <span style="color:green">Section 3.2: Preparing the testing set</span>

### As seen in previous sections, it is a best practice to clean the customers and stock test data sets from any elements which would not have been present in the training set. We covered a way to do this using SQL in previous sections. Below is a different way of getting the same result using Python/Spark transformations.

Run the cell below to clean up the testing set...

In [ ]:
# A Python set is an unordered collection of unique elements. We are building below two sets of
# customers and stock ids. The lambda function extracts the relevant column from the trainDf 
# dataframe, and we build a set which is then collected into a final array.
customers = set(trainDf.rdd.map(lambda line: line.custId).collect())
stock = set(trainDf.rdd.map(lambda line: line.stockCode).collect())

print testDf.count()
testDf = testDf.rdd.filter(lambda line: line.stockCode in stock and\
                                            line.custId in customers).toDF()

print testDf.count()

### The two numbers printed above correspond to the values obtained higher at the end of Section 1.7, which confirms the fact that all entries where a customer Id or stock Id was not present in the training set does not get a prediction rating value...

### <span style="color:green">Section 3.3: Getting predictions using the ML model with EXPLICIT feedback</span>

### Below is an excerpt from the Spark documentation regarding the "transform" method used in the cell below:
A Transformer is an abstraction that includes feature transformers and learned models. Technically, a Transformer implements a method transform(), which converts one DataFrame into another, generally by appending one or more columns. For example:

- A feature transformer might take a DataFrame, read a column (e.g., text), map it into a new column (e.g., feature vectors), and output a new DataFrame with the mapped column appended.
- A learning model might take a DataFrame, read the column containing feature vectors, predict the label for each feature vector, and output a new DataFrame with predicted labels appended as a column.

<span style="color:red">Note:</span> The original text from which this paragraph was copied can be found in the Spark documentation at the following link: http://spark.apache.org/docs/latest/ml-guide.html#transformers

Run the cell below to obtain predictions from both Spark ML explicit feedback models...

In [ ]:
# This transform method used below is therefore the method described in the second bullet above.
# It will take the test set as input, and apply the model predictions to it, appending a rating
# column and then return a new dataframe.
predictions1 = model1.transform(testDf)
predictions2 = model2.transform(testDf)

# A a quick verification, we will print the first couple of rows from both returned dataframes.
# We notice that the 'prediction' column was appended as expected.
print predictions1.take(2)
print predictions2.take(2)

### <span style="color:red">Remark:</span> In this case, it can be noted that the resulting predictions dataframes already have both the original customer rating, under the column 'preference' and the model's generated rating, under the column 'prediction'. It will therefore not be needed to construct a table or dataframe which contains both, as we already have it...

### <span style="color:green">Section 3.4: Comparing user entries with predicted ratings and getting a Mean Squared Error</span>

### The Mean Squared Error is evaluated in the same way as in previous sections.

Following examples from sections above, insert in the cell below two lines before the print statements to calculate the meanSquearedError1 and meanSquaredError2...
<div class="panel-group" id="accordion-46">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-46" href="#collapse1-46">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-46" class="panel-collapse collapse">
      <div class="panel-body"> 
meanSquaredError1 = predictions1.map(lambda line: (line.preference - line.prediction)\*\*2).mean()<br>
meanSquaredError2 = predictions2.map(lambda line: (line.preference - line.prediction)\*\*2).mean()
      </div>
    </div>
  </div>

In [ ]:

    
print 'Mean squared error = %.4f for our first model' % meanSquaredError1
print 'Mean squared error = %.4f for our second model' % meanSquaredError2

### The first model seems to present better accuracy than the second, which is expected given the chosen values for rank and maxIter...

### <span style="color:green">Section 3.5: Manual verification of ALS recommendations for one chosen customer...</span>

### We will now work on verifying "manually" the efficiency of our model. In order to do so, we will pick one customer who bought a reasonably small, yet meaningful number of items (i.e more than one or two, but not dozens) and check the top three recommendations for that particular customer in order to decide whether these make sense or not. To begin, we write a SQL query to find customers ids who bought a reasonable number of items, i.e between 5 and 10. We start with a count of 7 items (feel free to update the query below with a different count for other attemps)
#### <span style="color:red">Note:</span> We will continue using the same customer id as in previous section so as to compare outputs from different engines using the same baseline.

Use examples from previous sections to complete the cell below where you will look up customers who bought a reasonable (7?) number of items...
<div class="panel-group" id="accordion-47">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-47" href="#collapse1-47">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-47" class="panel-collapse collapse">
      <div class="panel-body"> 
select custId, count(*) as count from retailPurchases <br>
           group by custId having count = 7 order by count
      </div>
    </div>
  </div>

In [ ]:
query = """
"""

sqlContext.sql(query).take(10)

### Select customer id 16248...and check which items this customer bought...
<div class="panel-group" id="accordion-48">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-48" href="#collapse1-48">
        Hint for the cell below</a>
      </h4>
    </div>
    <div id="collapse1-48" class="panel-collapse collapse">
      <div class="panel-body"> 
SELECT distinct stockCode, description from retailPurchases <br>
               where custId in (16248)
      </div>
    </div>
  </div>

In [ ]:
sqlContext.sql("""  """).toPandas()

### We will now obtain some recommendations for the chosen customer through our selected model.
### Spark ML does not offer a 'recommendProducts' method as was used above with MLlib. But that is no problem, we will obtain our recommendations using the same 'transform' method which was used a few cells above with the test dataset. More specifically, we will:
- Build a dataframe named userItems, where our selected userid is associated with every single product id in the database
- Pass this userItems dataframe through the 'transform' method, which as seen previously, will append a 'rating' column, indicating the strength of the recommendation for our chosen user id and the current product
- Sort the resulting 'recommendations' dataframe and select the top N rows
<div class="panel-group" id="accordion-49">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-49" href="#collapse1-49">
        Hint 1</a>
      </h4>
    </div>
    <div id="collapse1-49" class="panel-collapse collapse">
      <div class="panel-body"> 
Select all the unique "stockCode" values from the trainDf dataframe as a single column and then augment this column with a second column (using withColumn) containing the literal "16248", or the customer of your choice for further processing.
      </div>
    </div>
  </div>
<div class="panel-group" id="accordion-50">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-50" href="#collapse1-50">
        Hint 2 (code)</a>
      </h4>
    </div>
    <div id="collapse1-50" class="panel-collapse collapse">
      <div class="panel-body"> 
userItems = trainDf.select("stockCode").distinct().\<br>
            withColumn('custId', lit(16248))
      </div>
    </div>
  </div>  

In [ ]:
# Build a dataframe named userItems, by selecting distinct values of all available stock codes
# and appending our chosen customer id.
from pyspark.sql.functions import lit


# Print a few rows from our dataframe to verify that it was built as expected.
print userItems.show(5)

### Use now 'transform' to rate each item. We will also sort the returned prediction codes in descending order of prediction strength, so as to see the highest recommendations first.
<div class="panel-group" id="accordion-51">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-51" href="#collapse1-51">
        Hint 1: run the transform on userItems</a>
      </h4>
    </div>
    <div id="collapse1-51" class="panel-collapse collapse">
      <div class="panel-body"> 
userItems = model1.transform(userItems)
      </div>
    </div>
  </div>
<div class="panel-group" id="accordion-52">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-52" href="#collapse1-52">
        Hint 2: print top 5 predictions. Requires sorting userItems on "prediction" column in descending order and picking top 5</a>
      </h4>
    </div>
    <div id="collapse1-52" class="panel-collapse collapse">
      <div class="panel-body"> 
print userItems.sort("prediction",ascending=False).show(5)
      </div>
    </div>
  </div>

In [ ]:
#execute the transform on userItems

#print userItems.take(5)


### We will use a now familiar SQL query to obtain the text description of the recommended items.

In [ ]:
query = """
SELECT 
    distinct stockCode, description 
FROM 
    retailPurchases 
WHERE 
    stockCode in (90214, 84596, 84997, 37489)
    ORDER BY stockCode limit 20
"""
items = sqlContext.sql(query)
print items.toPandas()

# <span style="color:blue">Section 4 (Please ensure you've run Section 0 above)</span>
### Use SPARK ML library and (assume) IMPLICIT user feedback on product ratings.
### SPARK ML uses data frames.

### <span style="color:green">Section 4.1: Training the implicit Spark ML models</span>

The cell below trains two Spark ML models with IMPLICIT feedback...

In [ ]:
from pyspark.ml.recommendation import ALS

als1 = ALS(rank=15, maxIter=15, implicitPrefs=True,userCol="custId",itemCol="stockCode",ratingCol="preference")
modelImplicit1ml = als1.fit(trainDf)

als2 = ALS(rank=2, maxIter=10,implicitPrefs=True, userCol="custId",itemCol="stockCode",ratingCol="preference")
modelImplicit2ml = als2.fit(trainDf)

print "The models have been trained"

### <span style="color:green">Section 4.2: Preparing the testing set</span>

### As seen in previous sections, it is a best practice to clean the customers and stock test data sets from any elements which would not have been present in the training set. We covered a way to do this using SQL in previous sections. Below is a different way of getting the same result using Python/Spark transformations.

Follow the example from Section 3 above to complete the cell below to clean up the testing set from entries which did not exist in the training set...
<div class="panel-group" id="accordion-53">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-53" href="#collapse1-53">
        Hint 1:</a>
      </h4>
    </div>
    <div id="collapse1-53" class="panel-collapse collapse">
      <div class="panel-body"> 
customers = set(trainDf.rdd.map(lambda line: line.custId).collect())<br>
stock = set(trainDf.rdd.map(lambda line: line.stockCode).collect())
      </div>
    </div>
  </div>
<div class="panel-group" id="accordion-54">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-54" href="#collapse1-54">
        Hint 2:</a>
      </h4>
    </div>
    <div id="collapse1-54" class="panel-collapse collapse">
      <div class="panel-body"> 
testDf = testDf.rdd.filter(lambda line: line.stockCode in stock and\ <br>
                                            line.custId in customers).toDF()
      </div>
    </div>
  </div>  

In [ ]:
XXXXXXX Code goes here (see Hint 1 above for help) XXXXXXX

print testDf.count()
XXXXXX Code goes here (see Hint 2 above for help) XXXXXX

print testDf.count()

### <span style="color:green">Section 4.3: Getting predictions using the ML model with IMPLICIT feedback</span>

### Below is an excerpt from the Spark documentation regarding the "transform" method used in the cell below:
A Transformer is an abstraction that includes feature transformers and learned models. Technically, a Transformer implements a method transform(), which converts one DataFrame into another, generally by appending one or more columns. For example:

- A feature transformer might take a DataFrame, read a column (e.g., text), map it into a new column (e.g., feature vectors), and output a new DataFrame with the mapped column appended.
- A learning model might take a DataFrame, read the column containing feature vectors, predict the label for each feature vector, and output a new DataFrame with predicted labels appended as a column.

<span style="color:red">Note:</span> The original text from which this paragraph was copied can be found in the Spark documentation at the following link: http://spark.apache.org/docs/latest/ml-guide.html#transformers

<div class="panel-group" id="accordion-55">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-55" href="#collapse1-55">
        Hint for the cell below:</a>
      </h4>
    </div>
    <div id="collapse1-55" class="panel-collapse collapse">
      <div class="panel-body"> 
customers = set(trainDf.rdd.map(lambda line: line.custId).collect())<br>
stock = set(trainDf.rdd.map(lambda line: line.stockCode).collect())
      </div>
    </div>
  </div>

In [ ]:
XXXXX Code goes here: Follow example from Section 3 above XXXXXX

### <span style="color:green">Section 4.4: Comparing user entries with predicted ratings and getting a Mean Squared Error</span>
<div class="panel-group" id="accordion-56">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-56" href="#collapse1-56">
        Hint for the cell below:</a>
      </h4>
    </div>
    <div id="collapse1-56" class="panel-collapse collapse">
      <div class="panel-body"> 
meanSquaredError1 = predictions1.map(lambda line: (line.preference - line.prediction)\*\*2).mean()<br>
meanSquaredError2 = predictions2.map(lambda line: (line.preference - line.prediction)\*\*2).mean()
      </div>
    </div>
  </div>  

In [ ]:
XXXXX Code goes here, follow examples from Section 3 above XXXXXXX
    
print 'Mean squared error = %.4f for our first model' % meanSquaredError1
print 'Mean squared error = %.4f for our second model' % meanSquaredError2

### The first model seems to present better accuracy than the second, which is expected given the chosen values for rank and maxIter...

### <span style="color:green">Section 4.5: Manual verification of ALS recommendations for one chosen customer...</span>

### We will now work on verifying "manually" the efficiency of our model. In order to do so, we will pick one customer who bought a reasonably small, yet meaningful number of items (i.e more than one or two, but not dozens) and check the top three recommendations for that particular customer in order to decide whether these make sense or not. To begin, we write a SQL query to find customers ids who bought a reasonable number of items, i.e between 5 and 10. We start with a count of 7 items (feel free to update the query below with a different count for other attemps)
#### <span style="color:red">Note:</span> We will continue using the same customer id as in previous section so as to compare outputs from different engines using the same baseline.

<div class="panel-group" id="accordion-57">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-57" href="#collapse1-57">
        Hint for the cell below:</a>
      </h4>
    </div>
    <div id="collapse1-57" class="panel-collapse collapse">
      <div class="panel-body"> 
query = """select custId, count(*) as count from retailPurchases <br>
           group by custId having count = 7 order by count"""<br><br>

sqlContext.sql(query).take(10)
      </div>
    </div>
  </div>  

In [ ]:
XXXXXX Code goes here, follow examples from Section 3 above XXXXXX

### Select customer id 16248...and check which items this customer bought...

<div class="panel-group" id="accordion-58">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-58" href="#collapse1-58">
        Hint for the cell below:</a>
      </h4>
    </div>
    <div id="collapse1-58" class="panel-collapse collapse">
      <div class="panel-body"> 
sqlContext.sql("""SELECT distinct stockCode, description from retailPurchases <br>
               where custId in (16248)""").toPandas()
      </div>
    </div>
  </div> 
 

In [ ]:
XXXXXX Code goes here, follow examples from Section 3 above XXXXXX

### We will now obtain some recommendations for the chosen customer through our selected model.
### Spark ML does not offer a 'recommendProducts' method as was used above with MLlib. But that is no problem, we will obtain our recommendations using the same 'transform' method which was used a few cells above with the test dataset. More specifically, we will:
- Build a dataframe named userItems, where our selected userid is associated with every single product id in the database
- Pass this userItems dataframe through the 'transform' method, which as seen previously, will append a 'rating' column, indicating the strength of the recommendation for our chosen user id and the current product
- Sort the resulting 'recommendations' dataframe and select the top N rows

<div class="panel-group" id="accordion-59">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-59" href="#collapse1-59">
        Hint 1:</a>
      </h4>
    </div>
    <div id="collapse1-59" class="panel-collapse collapse">
      <div class="panel-body"> 
This cell will be identical to the corresponding one in section 3.5 above.
      </div>
    </div>
</div>
<div class="panel-group" id="accordion-60">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-60" href="#collapse1-60">
        Hint 2 (shows code):</a>
      </h4>
    </div>
    <div id="collapse1-60" class="panel-collapse collapse">
      <div class="panel-body"> 
from pyspark.sql.functions import lit <br>

userItems = trainDf.select("stockCode").distinct().\ <br>
            withColumn('custId', lit(16248)) <br><br>

print userItems.show(5)
      </div>
    </div>
  </div> 

In [ ]:
# Get recommendations for one particular user

XXXXX Code goes here, follow example from Section 3 above XXXXXX


### Use now 'transform' to rate each item. We will also sort the returned prediction codes in descending order of prediction strength, so as to see the highest recommendations first.
<div class="panel-group" id="accordion-61">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-61" href="#collapse1-61">
        Hint for the cell below (shows code):</a>
      </h4>
    </div>
    <div id="collapse1-61" class="panel-collapse collapse">
      <div class="panel-body"> 
userItems = modelImplicit1ml.transform(userItems)<br>

#print userItems.take(5)<br>
userItems.sort("prediction",ascending=False).show(5)
      </div>
    </div>
  </div> 

In [ ]:
XXXXXX Code goes here, follow example from Section 3 above XXXXXXX

#print userItems.take(5)
XXXXXX Code goes here, follow example from Section 3 above XXXXXXX

### We will use a now familiar SQL query to obtain the text description of the recommended items.
<div class="panel-group" id="accordion-62">
 <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-62" href="#collapse1-62">
        Hint for the cell below (stockCode values may have to be adjusted):</a>
      </h4>
    </div>
    <div id="collapse1-62" class="panel-collapse collapse">
      <div class="panel-body"> 
SELECT <br>
    distinct stockCode, description <br>
FROM <br>
    retailPurchases <br>
WHERE <br>
    stockCode in (22111, 23355, 22866, 22865, 84029)<br>
    ORDER BY stockCode
      </div>
    </div>
  </div> 

In [ ]:
query = """
XXXXXX SQL statement goes here, follow examples from Section 3 above XXXXXX
"""
items = sqlContext.sql(query)
print items.toPandas()

### <span style="color:green">Section 4.6: Taking a look at the ALS internals...</span>

### The ALS model exposes the matrices resulting from the factorization as userFactors and ItemFactors.

### Let's take a look at the first few rows from the usersFactor matrix obtained from the second model.

In [ ]:
modelImplicit2ml.userFactors.take(5)

### Checking out the representation of the matrix, it is a DataFrame with an integer representing the Item and an array of double values representing the internal encoding of the item according to the chosen rank

In [ ]:
modelImplicit2ml.itemFactors

### The second model was generated with a rank value of 2, so the features array has two entries as expected.

In [ ]:
modelImplicit2ml.itemFactors.take(5)

### A few cells higher, we ran predictions using <span style="color:blue">modelImplicit1ml</span> for customer id <span style="color:red">16248</span> and product  <span style="color:red">22111</span>. We obtained a rating of <span style="color:red">0.114170134</span>. Let's attempt to reconstruct this result by manipulating the feature matrices directly.

### We will first extract the Vector representing user id 16248 from the userFactors matrix and print it. The vector has 15 components, since modelImplicit1 was built with a rank of 15.

In [ ]:
from pyspark.mllib.linalg import Vectors, DenseMatrix
from pyspark.mllib.linalg import DenseVector
from numpy import array
v1=Vectors.dense(modelImplicit1ml.userFactors.rdd.lookup(16248))
print v1[0]

### Second, we will extract the Vector representing product 22111 from the itemFactors matrix. This vector has 15 components as well.

In [ ]:
v2=DenseVector(modelImplicit1ml.itemFactors.rdd.lookup(22111))
print v2[0]

### Compute the dot product of both vectors.

In [ ]:
v1[0].dot(v2[0])

### Verify that you find the same value as the recommendation.

### <span style="color:red">Optional: Can you now implement the equivalent of "recommendProducts(userId, NumberOfRecommendations)" by manipulating the feature matrices and using dot products?</span>

##### Data Citation
Daqing Chen, Sai Liang Sain, and Kun Guo, Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining, Journal of Database Marketing and Customer Strategy Management, Vol. 19, No. 3, pp. 197â€“208, 2012 (Published online before print: 27 August 2012. doi: 10.1057/dbm.2012.17).